In [78]:
from PIL import Image
import numpy as np
import math
import random
from PIL import ImageOps
file = open('model_1.obj', 'r')
v = []
f = []
z_buffer = np.full((1000,1000), np.inf, dtype = np.float32)
img_mat = np.zeros((1000, 1000, 3), dtype = np.uint8)
for str in file:
    splitted_str = str.split()
    if(splitted_str[0] == 'v'):
        v.append((float(splitted_str[1]),float(splitted_str[2]),float(splitted_str[3])))
    if(splitted_str[0] == 'f'):
        f.append((int(splitted_str[1].split('/')[0]),int(splitted_str[2].split('/')[0]),int(splitted_str[3].split('/')[0])))
for i in range(len(f)):
        x0 = v[f[i][0]-1][0]* 5000 + 500
        y0 = v[f[i][0]-1][1]* 5000 + 250
        z0 = v[f[i][0]-1][2]* 5000 + 200
        x1 = v[f[i][1]-1][0]* 5000 + 500
        y1 = v[f[i][1]-1][1]* 5000 + 250
        z1 = v[f[i][1]-1][2]* 5000 + 200
        x2 = v[f[i][2]-1][0]* 5000 + 500
        y2 = v[f[i][2]-1][1]* 5000 + 250
        z2 = v[f[i][2]-1][2]* 5000 + 200
        
        n = normale(x0,y0,z0,x1,y1,z1,x2,y2,z2)
        cosA = scalar(n)
        if (cosA < 0):
            triangle_infill(x0,y0,x1,y1,x2,y2, img_mat, cosA, z0, z1, z2)
        # x_loop_finalll(img_mat, x0, y0, x1, y1)
        # x_loop_finalll(img_mat, x1, y1, x2, y2)
        # x_loop_finalll(img_mat, x2, y2, x0, y0)
img = Image.fromarray(img_mat, mode='RGB')
im = ImageOps.flip(img)
im.save("triangle_rabbit_with_trace_gradient.png")

In [4]:
#алгоритм Брезенхема
def x_loop_finalll(img_mat, x0, y0, x1, y1):
    flag = False
    if(abs(x0-x1) < abs(y0-y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        flag = True;
    if(x1<x0):
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    y = y0
    dy = 2 * abs((y1-y0))
    derror = 0
    y_update= 1 if y1 > y0 else -1
    for x in range(x0, x1):
        if(flag):
            img_mat[x,y,1] = 255
        else:
            img_mat[y,x,1] = 255
        derror += dy
        if (derror > (x1-x0)):
            derror -= 2 * (x1-x0)
            y += y_update

In [77]:

# canvas = np.zeros((1000, 1000, 3), dtype = np.uint8)

def triangle_infill(x0, y0, x1, y1, x2, y2, canvas, cosA, z0, z1, z2):
    # color0 = random.randint(0,255)
    # color1 = random.randint(0,255)
    # color2 = random.randint(0,255)
    
    xmin = min(x0, x1, x2)-1
    ymin = min(y0, y1, y2)-1
    xmax = max(x0, x1, x2)+1
    ymax = max(y0, y1, y2)+1
    
    if(xmin < 0):
        xmin = 0
    if(ymin < 0):
        ymin = 0
    for x in range(math.ceil(xmin), math.floor(xmax)):
        for y in range(math.ceil(ymin), math.floor(ymax)):
            lambd0, lambd1, lambd2 = bar_coord(x0, y0, x1, y1, x2, y2, x, y)
            if (lambd0 >= 0 and lambd1 >= 0 and lambd2 >= 0):
                canvas[y,x] = (-255 * cosA, -255 * cosA, -255 * cosA)
                # z_display= lambd0*z0 + lambd1*z1 + lambd2*z2
                # if(z_buffer[y,x] > z_display):
                #     canvas[y,x] = (-255 * cosA, -255 * cosA, -255 * cosA)
                #     z_buffer[y,x] = z_display

                
# bar_coord_2(-100,1,-100,100,1000,500)
# images = Image.fromarray(canvas, mode='RGB')
# images.save("triangle.png")

In [29]:
#Барицентрические координаты 1

def bar_coord(x0, y0, x1, y1, x2, y2, x, y):
    lambd0 = ((x - x2) * (y1 - y2) - (x1 - x2) * (y - y2)) /  ((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2)) 
    lambd1 = ((x0 - x2) * (y - y2) - (x - x2) * (y0 - y2)) /  ((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2)) 
    lambd2 = 1.0 - lambd0 - lambd1
    return lambd0, lambd1, lambd2


In [53]:
# вычисление нормали

def normale(x0,y0,z0,x1,y1,z1,x2,y2,z2):
    n = [0] * 3
    n[0] = ((y1-y2) * (z1-z0)) - ((z1-z2) * (y1-y0))
    n[1] = ((x1-x2) * (z1-z0)) - ((z1-z2) * (x1-x0))
    n[2] = ((x1-x2) * (y1-y0)) - ((y1-y2) * (x1-x0))
    return n

In [65]:
# скалярное произведение

def scalar(n):
    cosA = n[2]/ math.sqrt(n[0]**2 + n[1]**2 + n[2]**2)
    return cosA